<a href="https://colab.research.google.com/github/sayakpaul/A-B-testing-with-Machine-Learning/blob/master/Mini_Inception_EvoNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Which GPU?
!nvidia-smi

Sun Apr 19 10:01:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
# TensorFlow imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
# Other imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# Set the random seeds
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
# Set up wandb for easy experiment tracking
!pip install wandb -q
import wandb
wandb.login()

In [7]:
# Load and preprocess CIFAR10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
print(X_train.shape, X_test.shape)

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


## `EvoNorm2dB0`

In [0]:
# Reference
# https://github.com/lonePatient/EvoNorms_PyTorch/blob/master/models/normalization.py

def instance_std(x, eps=1e-5):
	# https://www.tensorflow.org/api_docs/python/tf/nn/moments
	_, var = tf.nn.moments(x, axes=[1, 2], keepdims=True)
	return tf.sqrt(var + eps)

class EvoNorm2dB0(tf.keras.layers.Layer):
	def __init__(self, in_channels, nonlinear=True, momentum=0.9,
		eps=1e-5):
		super(EvoNorm2dB0, self).__init__()
		self.nonlinear = nonlinear
		self.momentum = momentum
		self.eps = eps
		self.running_var = tf.ones((1, 1, 1, in_channels))

		def build(self):
			self.gamma = self.add_variable("gamma",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Ones())
			self.beta = self.add_variable("beta",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Zeros())
			if self.nonlinear:
				self.v = self.add_variable("v",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Ones())

		def call(self, x):
			N, H, W, C = tf.shape(x)

			if self.training:
				x1 = tf.transpose(x, [3, 0, 1, 2])
				x1 = tf.reshape(x1, (C, -1))
				var = tf.math.square(tf.math.reduce_std(x1, axis=1))
				var = tf.reshape(var, (1, 1, 1, C))
				self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var
			else:
				var = self.running_var

			if self.nonlinear:
				den = tf.math.maximum(tf.sqrt(var), self.v * x + instance_std(x))
				return x / den * self.gamma + self.beta
			else:
				return x * self.gamma + self.beta

## `EvoNorm2dS0`

In [0]:
# Reference
# https://github.com/lonePatient/EvoNorms_PyTorch/blob/master/models/normalization.py

def group_std(x, groups=32, eps=1e-5):
	N, H, W, C = tf.shape(x)
	x = tf.reshape(x, [N, H, W, groups, C // groups])
	_, var = tf.nn.moments(x, [1, 2, 4], keepdims=True)
	std = tf.sqrt(var + eps)
	std = tf.broadcast_to(std, x.shape)
	return tf.reshape(std, (N, H, W, C))

class EvoNorm2dS0(tf.keras.layers.Layer):
	def __init__(self, in_channels, groups=32, nonlinear=True):
		super(EvoNorm2dS0, self).__init__()
		self.nonlinear = nonlinear
		self.groups = groups

		def build(self):
			self.gamma = self.add_variable("gamma",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Ones())
			self.beta = self.add_variable("beta",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Zeros())
			if self.nonlinear:
				self.v = self.add_variable("v",
									shape=(1, 1, 1, self.in_channels),
									initializer=tf.initializers.Ones())

		def call(self, x):
			if self.nonlinear:
				num = x * tf.nn.sigmoid(self.v * x)
				return num / group_std(x) * self.gamma + self.beta
			else:
				return x * self.gamma + self.beta

## Mini Inception

In [0]:
# Implementation comes from http://pyimg.co/mac01
def minigooglenet_functional(width, height, depth, classes, norm=EvoNorm2dB0, groups=32):
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same"):
		# define a CONV => EvoNorm pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding)(x)
		
		if isinstance(norm, EvoNorm2dS0):
			layer = norm(in_channels=K, groups=groups)
		else:
			layer = norm(in_channels=K)
		
		x = layer(x)

		# return the block
		return x

	def inception_module(x, numK1x1, numK3x3, chanDim):
		# define two CONV modules, then concatenate across the
		# channel dimension
		conv_1x1 = conv_module(x, numK1x1, 1, 1, (1, 1), chanDim)
		conv_3x3 = conv_module(x, numK3x3, 3, 3, (1, 1), chanDim)
		x = concatenate([conv_1x1, conv_3x3], axis=chanDim)

		# return the block
		return x

	def downsample_module(x, K, chanDim):
		# define the CONV module and POOL, then concatenate
		# across the channel dimensions
		conv_3x3 = conv_module(x, K, 3, 3, (2, 2), chanDim,
			padding="valid")
		pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
		x = concatenate([conv_3x3, pool], axis=chanDim)

		# return the block
		return x

	# initialize the input shape to be "channels last" and the
	# channels dimension itself
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input and first CONV module
	inputs = Input(shape=inputShape)
	x = conv_module(inputs, 96, 3, 3, (1, 1), chanDim)

	# two Inception modules followed by a downsample module
	x = inception_module(x, 32, 32, chanDim)
	x = inception_module(x, 32, 48, chanDim)
	x = downsample_module(x, 80, chanDim)

	# four Inception modules followed by a downsample module
	x = inception_module(x, 112, 48, chanDim)
	x = inception_module(x, 96, 64, chanDim)
	x = inception_module(x, 80, 80, chanDim)
	x = inception_module(x, 48, 96, chanDim)
	x = downsample_module(x, 96, chanDim)

	# two Inception modules followed by global POOL and dropout
	x = inception_module(x, 176, 160, chanDim)
	x = inception_module(x, 176, 160, chanDim)
	x = AveragePooling2D((7, 7))(x)
	x = Dropout(0.5)(x)

	# softmax classifier
	x = Flatten()(x)
	x = Dense(classes)(x)
	x = Activation("softmax")(x)

	# create the model
	model = Model(inputs, x, name="minigooglenet")

	# return the constructed network architecture
	return model

In [0]:
# One-hot encoding of the labels
y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_test_ohe = tf.keras.utils.to_categorical(y_test)

In [0]:
# Hyperparameters
BATCH_SIZE=128
EPOCHS=60

In [0]:
# Import wandb's Keras callback
from wandb.keras import WandbCallback

## With `EvoNorm2dB0` and no data agumentation

In [14]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dB0-no-aug-crctd")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 57s 145ms/step - loss: nan - accuracy: 0.1016 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
391/391 [==============================] - 55s 141ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
391/391 [=======

In [0]:
# Let's try with data augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=18, 
    zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, 
    shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

## With `EvoNorm2dB0` and data agumentation

In [16]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dB0-data-aug-crctd")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 23s 60ms/step - loss: nan - accuracy: 0.1018 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
390/390 [==============================] - 22s 57ms/step - loss: nan - accuracy: 0.1023 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
390/390 [==============================] - 22s 57ms/step - loss: nan - accuracy: 0.0989 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
390/390 [==============================] - 22s 57ms/step - loss: nan - accuracy: 0.0994 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
390/390 [==============================] - 22s 57ms/step - loss: nan - accuracy: 0.1014 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
390/390 [==============================] - 22s 57ms/step - loss: nan - accuracy: 0.0993 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
390/390 [==============================] - 22s 56ms/step - loss: nan - accuracy: 0.0968 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
390/390 [==============

## With `EvoNorm2dS0` and no data augmentation (groups of 8)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-no-aug-group8")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=8)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - ETA: 0s - loss: 1.7933 - accuracy: 0.3336

wandb: ERROR Can't save model, h5py returned error: Layer EvoNorm2dS0 has arguments in `__init__` and therefore must override `get_config`.


391/391 [==============================] - 16s 40ms/step - loss: 1.7933 - accuracy: 0.3336 - val_loss: 1.5026 - val_accuracy: 0.4582
Epoch 2/60
391/391 [==============================] - 15s 39ms/step - loss: 1.3575 - accuracy: 0.5188 - val_loss: 1.1937 - val_accuracy: 0.5863
Epoch 3/60
391/391 [==============================] - 15s 39ms/step - loss: 1.1718 - accuracy: 0.5894 - val_loss: 1.1102 - val_accuracy: 0.6220
Epoch 4/60
391/391 [==============================] - 15s 39ms/step - loss: 1.0431 - accuracy: 0.6382 - val_loss: 0.9702 - val_accuracy: 0.6624
Epoch 5/60
391/391 [==============================] - 15s 39ms/step - loss: 0.9684 - accuracy: 0.6662 - val_loss: 0.9526 - val_accuracy: 0.6762
Epoch 6/60
391/391 [==============================] - 15s 39ms/step - loss: 0.9099 - accuracy: 0.6873 - val_loss: 0.9356 - val_accuracy: 0.6764
Epoch 7/60
391/391 [==============================] - 15s 39ms/step - loss: 0.8440 - accuracy: 0.7090 - val_loss: 0.8492 - val_accuracy: 0.7110
Epo

## With `EvoNorm2dS0` and no data augmentation (groups of 16)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-no-aug-group16")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=16)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - ETA: 0s - loss: 1.8308 - accuracy: 0.3224

wandb: ERROR Can't save model, h5py returned error: Layer EvoNorm2dS0 has arguments in `__init__` and therefore must override `get_config`.


391/391 [==============================] - 16s 40ms/step - loss: 1.8308 - accuracy: 0.3224 - val_loss: 1.5047 - val_accuracy: 0.4589
Epoch 2/60
391/391 [==============================] - 15s 39ms/step - loss: 1.3696 - accuracy: 0.5099 - val_loss: 1.2704 - val_accuracy: 0.5501
Epoch 3/60
391/391 [==============================] - 15s 39ms/step - loss: 1.1897 - accuracy: 0.5844 - val_loss: 1.1100 - val_accuracy: 0.6219
Epoch 4/60
391/391 [==============================] - 15s 39ms/step - loss: 1.0697 - accuracy: 0.6273 - val_loss: 1.0316 - val_accuracy: 0.6340
Epoch 5/60
391/391 [==============================] - 15s 39ms/step - loss: 0.9876 - accuracy: 0.6610 - val_loss: 0.9818 - val_accuracy: 0.6625
Epoch 6/60
391/391 [==============================] - 15s 39ms/step - loss: 0.9135 - accuracy: 0.6849 - val_loss: 0.9120 - val_accuracy: 0.6852
Epoch 7/60
391/391 [==============================] - 15s 39ms/step - loss: 0.8565 - accuracy: 0.7054 - val_loss: 0.8661 - val_accuracy: 0.7071
Epo

## With `EvoNorm2dS0` with no data augmentation (groups of 32)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-no-aug-group32")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=32)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 16s 40ms/step - loss: nan - accuracy: 0.1014 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
391/391 [==============================] - 15s 39ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
391/391 [==============

## With `EvoNorm2dS0` with data augmentation (groups of 8)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-data-aug-group8")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=8)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0980 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.1024 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.1044 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.0970 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.0990 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.0970 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.1028 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
390/390 [==============

## With `EvoNorm2dS0` with data augmentation (groups of 16)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-data-aug-group16")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=16)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 12s 32ms/step - loss: nan - accuracy: 0.0989 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1023 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1002 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0983 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0980 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
390/390 [==============================] - 12s 32ms/step - loss: nan - accuracy: 0.1025 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1006 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
390/390 [==============

## With `EvoNorm2dS0` with data augmentation (groups of 32)

In [0]:
# Initialize wandb
wandb.init(project="EvoNorm-TensorFlow2", id="EvoNorm2dS0-data-aug-group32")

# Optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train the model
model = minigooglenet_functional(32, 32, 3, 10, norm=EvoNorm2dS0, groups=32)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1027 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0967 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1014 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.0988 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1009 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/60
390/390 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.0949 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/60
390/390 [==============================] - 12s 30ms/step - loss: nan - accuracy: 0.1059 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/60
390/390 [==============